# 0-application-walk-thourgh

In this notebook, we introduce the basics of PyLipID via a walk-through application. We keep the details to minimum and invite users to refer to other specific notebooks in which you can find more details, exercise and theory. 

In this tutorial, we show the PyLipID workflow of checking the lipid interactions a Class A GPCR, Adenosine A2a receptor from coarse-grained simulations with a complex membrane bilayer that contains 7 lipid species. 

<img src="_statics/A2aR_system_overview.pdf" width="520" height="280" />

The simulation ensemble reported in the original paper contained 10 repeats, but here we took 2 of the repeats for demonstration and saved in two directories ``run1`` and ``run2``, both of which contains a simulation trajectory ``protein_lipids.xtc`` and a corresponding topology coordinate ``protein_lipids.gro``. The two simulation trajectories have the same topology but different initial configurations. The trajectories were saved with a 20 ns timestep. 

In [1]:
%matplotlib inline
import pylipid
from pylipid.api import LipidInteraction

## Load data

We start by initializing the main class `LipidInteraction`. 

Trajectories and topology coordinates of the two repeats were saved in the directory ``../../tests/data``. Thus we first create a `trajfile_list` and `topfile_list`. Then, we load the `trajfile_list` and `topfile_list` to `LipidInteraction` together with a couple of calculation settings, including what cutoffs to use, which lipid to check, which lipid atoms are used to define the interactions and what time unit the generated data use etc. You can also specify which directory the generated data should be saved at. But here we will use the default value, i.e. the current working directory. 

In [2]:
trajfile_list = ["../../tests/data/run1/protein_lipids.xtc", "../../tests/data/run2/protein_lipids.xtc"]
topfile_list = ["../../tests/data/run1/protein_lipids.gro", "../../tests/data/run2/protein_lipids.gro"]
lipid = "CHOL"
cutoffs = [0.55, 0.8]
nprot = 1
timeunit = 'us' # micro-second

# initialize 
li = LipidInteraction(trajfile_list, topfile_list, cutoffs=cutoffs, lipid=lipid, nprot=nprot)

Creating new director: /Users/lindasong/PyLipID/PyLipID-dev/docs/tutorials/Interaction_CHOL


## Collect interactions for residues

Calculate interaction durations, occupancy and lipid count from trajectories. The correlation coefficient matrix will also be calculated from these interactions. A pandas.DataFrame object will be created to store the generated data and be stored as a class instance object `dataset`. 

In [3]:
li.collect_residue_contacts(write_log=True, print_log=False)

# print_log=True will print out the residues with the highest interaction durations, interaction 
# occupancy and lipid count from each trajectory. 

# write_log=True will write such information in a log file. 

COLLECT INTERACTIONS FROM TRAJECTORIES: 100%|██████████| 2/2 [00:48<00:00, 24.10s/it]


After calculation, a couple of class instance objects are generated. For example, the instance object `durations` stores the durations of lipid interaction with protein residues in a dictionary object. 

In [4]:
residue_id = 20
print(li.durations[residue_id])

[[0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.04, 0.04, 0.04, 0.16, 0.26], [0.02, 0.02, 0.02, 0.02, 0.04, 0.06, 0.08, 0.1, 0.12, 0.14, 0.18, 0.26, 0.38]]


It shows cholesterol interaction with the 21th residue (index starting from 0) in a list of two lists, each of which contains interactions durations from the corresponding trajectory in `trajfile_list`. 

A class instance object `dataset` is also generated, which stores the averages of various interaction metrics for each residues. The information in this `dataset` will be expanded to include that of the binding site after the calculation of binding site interaction has been carried out. 

In [5]:
li.dataset.head()

,Residue,Residue ID,Occupancy,Occupancy std,Duration,Duration std,Lipid Count,Lipid Count std
0,1ILE,0,48.007968,2.191235,0.040816,0.033473,1.087129,0.000173
1,2MET,1,14.143426,0.597610,0.030566,0.021493,1.042925,0.015898
2,3GLY,2,6.374502,0.000000,0.037895,0.038331,1.000000,0.000000
3,4SER,3,1.992032,1.992032,0.048571,0.070797,0.500000,0.500000
4,5SER,4,48.207171,8.366534,0.056757,0.090585,1.077817,0.027817


The `LipidInteraction` class also has a couple of assisting functions that provide quick access to the class attributes. 

## Calculate koff/residence time for residues

We next move to calculate the interaction koff and residence time for protein residues. This calculation has to be run after ``collect_residue_contacts()`` as the koff is calculated from interaction durations. 

In [6]:
koffs, res_times = li.compute_residue_koff(print_data=False, plot_data=True)

# if you are interested in only a couple of residues or one specific residue, you can provide a list
# of residue id or one residue id as follow:
# li.compute_residue_koff(residue_id=[10,15,25,35])
# li.compute_residue_koff(residue_id=10)

Creating new director: /Users/lindasong/PyLipID/PyLipID-dev/docs/tutorials/Interaction_CHOL/Residue_koffs_CHOL


`print_data=True` will print out the calculated interactions for the residues. It can be quite verbose.

`plot_data=True` will plot the koff figure for the residues. The directory to save such figures can be changed by save_dir, otherwise they will be saved in the directory of the Residue_koff_{lipid} at the current working directory. 

The koff figure looks like this:

<img src="_statics/koff_figure.pdf" />

The left panel plots the interactions durations in the sorted order and the right panel plots the normalised survival rates of these interactions (the purple doted line) and the bi-exponential curve fitted to survival curve (red broken line). To check the sampling quality, the fitted curve of the bootstraped interaction durations are plotted in gray lines. 

## Calculate binding sites

Before calculation of interactions with binding sites, we need to calculate where the binding sites are. 

In [7]:
note_list = li.compute_binding_nodes(threshold=4, print_data=False)

# threshold=4 decides that binding sites should contain at least 4 residues. 
# print_data=True will print the residues for each binding site. It can be quite verbose. 

In [19]:
print(node_list)

NameError: name 'node_list' is not defined

This function returns a node list, which is also retored in the class instance object node_list. The node list is a list of lists, each of which contains residues for a binding site.  

## Calculate interactions with binding sites

Once binding sites are defined, we can move to the calculation of interactions with binding sites. 

In [8]:
li.compute_site_koff(print_data=False, plot_data=True, sort_residue="Residence Time")

CALCULATE INTERACTIONS FOR BINDING SITES:   0%|          | 0/10 [00:00<?, ?it/s]

Creating new director: /Users/lindasong/PyLipID/PyLipID-dev/docs/tutorials/Interaction_CHOL/Binding_Sites_koffs_CHOL


CALCULATE INTERACTIONS FOR BINDING SITES: 100%|██████████| 10/10 [00:06<00:00,  1.65it/s]


([2.8865742738076983,
  2.389003330062132,
  10.055461689621868,
  2.2030557179316186,
  4.743122424101136,
  7.176946463458787,
  7.084518233188024,
  7.24927898237628,
  3.3540386572212073,
  1.2962624878052542],
 [0.34643141147409096,
  0.4185845986133441,
  0.09944844213688259,
  0.4539149835660395,
  0.21083158109491743,
  0.13933502292255778,
  0.14115285854095422,
  0.13794475318595129,
  0.298148024575391,
  0.7714486914553346])

`print_data=True` will print out information for binding sites in which the comprising residues are sorted based on the value defined by `sort_residue`. Such information can be written out in a log file by `li.write_site_info()`. Similar to `compute_residue_koff()`, `compute_site_koff` also can plot the koff figure for binding sites, controled by `plot_data`. Specific binding sites can also be selected for calculated via `binding_site_id`. 

In [9]:
koff_bs, res_time_bs = \
li.compute_site_koff(binding_site_id=2, print_data=True, plot_data=False, sort_residue="Residence Time")

CALCULATE INTERACTIONS FOR BINDING SITES: 100%|██████████| 1/1 [00:00<00:00,  2.84it/s]

# Binding site 2
 Binding Site Residence Time:       0.099 us   
 Binding Site Koff:                10.055  R squared:  0.9997
 Binding Site Duration:             0.057 us   
 Binding Site Occupancy:           94.024 %
 Binding Site Lipid Count:          2.163
 Pos. Charge:        4/17      
 Neg. Charge:        0/17      
 Polar:              1/17      
 Hydrophobic:        10/17     
 Special:            2/17      
 Residue Res ID  Res. Time (us)  Duration (us)   Occupancy (%) Lipid Count  Koff  R Squared 
  21GLY    20        0.218           0.078          15.538        1.010     4.598   0.998   
  24LEU    23        0.164           0.113          50.598        1.074     6.081   1.000   
 302ARG    301       0.149           0.045          39.044        1.062     6.702   1.000   
 294ARG    293       0.145           0.044           7.570        1.000     6.882   0.999   
  27TRP    26        0.120           0.075          48.207        1.103     8.361   0.999   
 300ILE    299       

The binding site information can also be viewed by the class attribute:

In [10]:
df = li.binding_site(binding_site_id=0)

# Binding site 0
 Binding Site Residence Time:       0.346 us   
 Binding Site Koff:                 2.887  R squared:  0.9987
 Binding Site Duration:             0.087 us   
 Binding Site Occupancy:           98.008 %
 Binding Site Lipid Count:          2.570
 Pos. Charge:        0/15      
 Neg. Charge:        1/15      
 Polar:              2/15      
 Hydrophobic:        11/15     
 Special:            1/15      
 Residue Res ID  Res. Time (us)  Duration (us)   Occupancy (%) Lipid Count  Koff  R Squared 
  5SER      4        0.385           0.057          48.207        1.078     2.598   0.995   
 269TYR    268       0.370           0.230           1.992        0.500     2.701   0.973   
  14ILE    13        0.262           0.084          65.538        1.052     3.818   0.999   
  10VAL     9        0.241           0.073          83.665        1.326     4.147   0.999   
  7TYR      6        0.241           0.080          50.398        1.059     4.154   0.997   
 273VAL    272       

The returned `df` is a pandas.DataFrame object that contains the residues of the selected binding site and their interaction data. 

In [11]:
df.head()

,Residue,Residue ID,Occupancy,Occupancy std,Duration,Duration std,Lipid Count,Lipid Count std,Koff,Residence Time,R Squared,Koff Bootstrap avg,Binding Site ID,Binding Site Koff,Binding Site Residence Time,Binding Site R Squared,Binding Site Duration,Binding Site Occupancy,Binding Site Lipid Count
0,1ILE,0,48.007968,2.191235,0.040816,0.033473,1.087129,0.000173,27.501646,0.036361,0.999844,26.424952,0.0,2.886574,0.346431,0.998745,0.086627,98.007968,2.569561
1,2MET,1,14.143426,0.597610,0.030566,0.021493,1.042925,0.015898,30.874937,0.032389,0.999933,33.770820,0.0,2.886574,0.346431,0.998745,0.086627,98.007968,2.569561
2,3GLY,2,6.374502,0.000000,0.037895,0.038331,1.000000,0.000000,17.343492,0.057659,0.997950,18.580244,0.0,2.886574,0.346431,0.998745,0.086627,98.007968,2.569561
4,5SER,4,48.207171,8.366534,0.056757,0.090585,1.077817,0.027817,2.597775,0.384945,0.995299,14.165048,0.0,2.886574,0.346431,0.998745,0.086627,98.007968,2.569561
5,6VAL,5,65.737052,0.398406,0.055250,0.056369,1.124192,0.008338,14.076645,0.071040,0.999795,15.069725,0.0,2.886574,0.346431,0.998745,0.086627,98.007968,2.569561


## Analyze bound poses

Once the binding sites are defined, we can also proceed to analyze bound poses, e.g. check the most representative bound poses for binding sites and check bound pose clusters for binding sites etc. 

In [12]:
pose_pool, pose_rmsd = li.analyze_bound_poses()

Creating new director: /Users/lindasong/PyLipID/PyLipID-dev/docs/tutorials/Interaction_CHOL/Bound_Poses_CHOL


ANALYZE BOUND POSES: 100%|██████████| 10/10 [00:10<00:00,  1.09s/it]


`analyze_bound_poses()` will rate the bound poses for each binding site based on probability density calculated from the simulation trajectories and write out a couple of top rated poses (controled by `n_poses`). It will also cluster the bound poses for each binding site, for which the number of clusters can be determined by a density based cluster `DBSCAN` or provided via `n_clusters`. The lipid bound poses for each binding site will be returned. The RMSD of bound poses to the binding site average will be returned and plotted in a violin plot. 

<img src="_statics/Pose_RMSD_violinplot.pdf" width="620" height="380" />

Similar to other analysis functions, `analyze_bound_poses` can also select a couple of binding sites for analysis via `binding_site_id`. 

The returned pose_pool is a `dict` object that stores bound poses for binding sites. psoe_rmsd is a pandas.DataFrame that stores RMSD for the bound poses (in unit of nm).

In [13]:
pose_rmsd.head()

,Binding Site 0,Binding Site 1,Binding Site 2,Binding Site 3,Binding Site 4,Binding Site 5,Binding Site 6,Binding Site 7,Binding Site 8,Binding Site 9
0,0.398494,0.453347,0.480430,0.414447,0.477954,0.223619,0.641076,0.491868,0.501663,0.407428
1,0.308050,0.508066,0.586139,0.483510,0.548540,0.519717,0.514189,0.514480,0.545599,0.522759
2,0.320346,0.559859,0.430262,0.439258,0.763437,0.230730,0.644078,0.700336,0.682936,0.712557
3,0.305909,0.505883,0.286762,0.434507,0.586024,0.388090,0.530072,0.372596,0.501795,0.728238
4,0.382212,0.421467,0.742836,0.410406,0.551884,0.170766,0.375149,0.475495,0.474173,0.362353


## Calculate surface area

We can also calculate the surface area for binding site, obtaining the values as a function of time. 

In [14]:
surface_area = li.compute_surface_area(plot_data=True)

CALCULATE SURFACE AREA PER TRAJ: 100%|██████████| 2/2 [00:19<00:00,  9.89s/it]


The returned `surface_area` is pandas.DataFrame object that stores the time evolution of the surface area for each binding site. The reported values are in the unit of nm^2.

In [15]:
surface_area.head()

Binding Site 0  Binding Site 1  Binding Site 2  Binding Site 3  \
0 0 0        9.831926        6.162645       14.217576        8.106370   
    1        9.532832        6.518453       13.996737        8.952161   
    2        9.920264        5.824062       14.378225        8.116406   
    3        9.925804        6.969460       13.559234        8.680886   
    4        9.200138        6.094445       14.480817        9.937469   

       Binding Site 4  Binding Site 5  Binding Site 6  Binding Site 7  \
0 0 0       21.574053        4.319067        8.400154       19.111601   
    1       21.588299        4.317808        8.342923       19.478838   
    2       21.076975        4.325853        8.756776       19.252750   
    3       21.540623        4.396411        9.086758       19.427549   
    4       21.841776        4.352310        8.739532       18.872972   

       Binding Site 8  Binding Site 9  Time  
0 0 0        9.822882       10.777733  3.00  
    1       10.775518       10.718760  3.02  
    2       10.844900       10.001993  3.04  
    3        9.927571       10.028803  3.06  
    4       10.298021        9.405209  3.08

`print_data=True` generates a timeseries plot and a violinplot. 

<img src="_statics/Surface_Area_timeseries.pdf" />

<img src="_statics/Surface_Area_violinplot.pdf" />

Similar to other binding site analysis functions, `compute_surface_area` allows for selection of binding site for calculation via `binding_site_id`.


The calculation of surface area needs definition of the radii of atoms in the system. The function uses `mdtraj`'s atom radii and defines the radii of MARTINI 2 beads. New radii definition can be provided via `radii`. 

## Plots and save data

The `LipidInteraction` provides a couple of assisting functions to plot and save data. 

In [16]:
for item in ["Residence Time", "Duration", "Occupancy", "Lipid Count"]:
    li.plot(item=item) # plot values as a function of time

Creating new director: /Users/lindasong/PyLipID/PyLipID-dev/docs/tutorials/Interaction_CHOL/Figure_CHOL


<img src="_statics/Residence_Time.pdf" width="520" height="380" />

<img src="_statics/Duration.pdf" width="520" height="380" />

<img src="_statics/Occupancy.pdf" width="520" height="380" />

<img src="_statics/Lipid_Count.pdf" width="520" height="380" />

In [17]:
li.plot_logo(item="Residence Time")

<img src="_statics/Residence_Time_logo.pdf" width="320" height="380" />

In [18]:
li.write_site_info(sort_residue="Duration") # write binding site information in a txt file

li.save_data(item="Dataset") # save the pandas.DataFrame object dataset in a csv file

li.save_data(item="Duration") # save all the interaction durations in pickle

li.save_coordinate(item="Residence Time") # write protein coordinate in pdb format with the b factor column
                                          # showing 'Residence Time' values

li.save_pymol_script(pdb_file="../../tests/data/receptor.pdb") # write a pymol script that maps binding 
                                                               # site information to protein structure. 

Creating new director: /Users/lindasong/PyLipID/PyLipID-dev/docs/tutorials/Interaction_CHOL/Dataset_CHOL
Creating new director: /Users/lindasong/PyLipID/PyLipID-dev/docs/tutorials/Interaction_CHOL/Coordinate_CHOL
